<a href="https://colab.research.google.com/github/zhuqiwen/e533-deep-learning-systems-2023-fall/blob/main/hw1/1-pytorch_softmax_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [12]:
# define class and functions

# Softmax class
class Softmax(nn.Module):

  def __init__(self, inputLength, outputLength):
    super(Softmax, self).__init__()
    self.linear = nn.Linear(inputLength, outputLength)
    self.optimizer = optim.SGD(self.parameters(), lr=0.01)
    self.lossFn = nn.CrossEntropyLoss()
    self.batchSize = 16


  def forward(self, x):
    return self.linear(x)

# prepare
def getData():
  data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
  data = list(data)
  trainLength = int(len(data) * 0.8)
  return (data[:trainLength], data[trainLength:])


trainData, validateData = getData()

len(trainData)


48000